In [31]:
from utils import *
import gc
import numpy as np
import math

In [2]:
db, engine, meta = db_connect()

In [3]:
tables = db_tables(meta)

In [ ]:
tables

In [4]:
leg_map = get_leg_map(engine)

In [ ]:
len(leg_map)

In [ ]:
list(leg_map.values())[0]

In [ ]:
# make congresses a map for easier parsing
for key, leg in leg_map.items():
    congress_map = {}
    for congress in leg["congresses"]:
        congress_map[congress["congress"]] = congress
    leg["congress_map"] = congress_map

In [ ]:
update_all(engine, tables["leg"], leg_map, verbose = True, increment = 100)

In [ ]:
all_bills = get_bills_map(engine)

In [ ]:
i = 0
total = len(leg_map)
for key, leg in leg_map.items():
    congress_map = leg["congress_map"]
    for congress in congress_map.values():
        congress["bills_sponsored"] = 0
        congress["total_cosponsors"] = 0
        congress["bills_cosponsored"] = len([bill for bill in leg["cosponsored_bills"] if bill["congress"] == congress["congress"]])
    for bill in leg["sponsored_bills"]:
        congress = congress_map[bill["congress"]]
        congress["bills_sponsored"] += 1
        full_bill = all_bills[bill["id"]]
        congress["total_cosponsors"] += len(full_bill["cosponsors"])
    for congress in congress_map.values():
        if (congress["bills_sponsored"] != 0):
            congress["cosponsors_per_bill"] = congress["total_cosponsors"] / congress["bills_sponsored"]
            congress["cosponsored_sponsored_ratio"] = congress["bills_cosponsored"] / congress["bills_sponsored"]
    i += 1
    if i % 100 == 0:
        print("Finished {} of {}".format(i, total))
        

In [ ]:
leg_map = get_leg_map(engine)

In [7]:
congress_maps = [leg["congress_map"] for leg in leg_map.values()]

In [20]:
def get_congress_vals(congress_maps, congress_no, chamber, field):
    congress_str = str(congress_no)
    return [c[congress_str][field] for c in congress_maps if congress_str in c and c[congress_str]["chamber"] == chamber and field in c[congress_str]]

In [27]:
congress_stats = None

In [44]:
congress_agg = {}

for i in range(93, 116):
    congress_agg[i] = {
        "house": {
            "bills_sponsored" : [], 
            "bills_cosponsored" : [],
            "cosponsors_per_bill" : [], 
            "cosponsored_sponsored_ratio" : []
        },
        "senate" : {
            "bills_sponsored" : [], 
            "bills_cosponsored" : [],
            "cosponsors_per_bill" : [], 
            "cosponsored_sponsored_ratio" : []
        }
    }
    for chamber_key, chamber_val in congress_agg[i].items():
        for field in chamber_val.keys():
            chamber_val[field] = get_congress_vals(congress_maps, i, chamber_key, field)

In [29]:
congress_agg[93]

{'house': {'bills_sponsored': [30,
   33,
   57,
   40,
   61,
   64,
   30,
   136,
   42,
   112,
   29,
   34,
   8,
   8,
   72,
   42,
   19,
   16,
   51,
   18,
   40,
   3,
   105,
   142,
   31,
   22,
   12,
   51,
   17,
   149,
   48,
   59,
   76,
   38,
   36,
   11,
   51,
   2,
   39,
   14,
   71,
   4,
   16,
   47,
   48,
   18,
   46,
   36,
   35,
   21,
   15,
   34,
   74,
   3,
   53,
   40,
   58,
   12,
   62,
   2,
   107,
   38,
   50,
   12,
   35,
   29,
   107,
   59,
   58,
   112,
   14,
   74,
   8,
   51,
   39,
   6,
   67,
   51,
   129,
   2,
   20,
   27,
   26,
   55,
   54,
   3,
   136,
   1,
   40,
   21,
   5,
   22,
   5,
   32,
   32,
   87,
   28,
   32,
   17,
   73,
   29,
   46,
   199,
   35,
   11,
   97,
   77,
   41,
   94,
   21,
   59,
   23,
   7,
   23,
   86,
   26,
   28,
   14,
   30,
   18,
   69,
   11,
   19,
   20,
   55,
   35,
   5,
   45,
   8,
   20,
   19,
   66,
   20,
   24,
   13,
   40,
   2,
   77,
   53,
   42,

In [14]:
get_congress_vals(congress_maps, 100, "house", "bills_sponsored")

[3,
 4,
 4,
 10,
 5,
 18,
 5,
 14,
 9,
 3,
 0,
 28,
 12,
 16,
 7,
 2,
 38,
 28,
 30,
 12,
 24,
 30,
 12,
 6,
 20,
 10,
 0,
 45,
 3,
 36,
 22,
 0,
 4,
 15,
 10,
 11,
 10,
 5,
 35,
 18,
 9,
 6,
 11,
 8,
 12,
 11,
 8,
 12,
 5,
 18,
 4,
 29,
 8,
 7,
 9,
 20,
 7,
 21,
 23,
 3,
 1,
 19,
 11,
 10,
 26,
 17,
 17,
 3,
 6,
 39,
 1,
 4,
 8,
 6,
 53,
 13,
 3,
 17,
 14,
 13,
 5,
 7,
 27,
 37,
 8,
 10,
 9,
 5,
 1,
 2,
 2,
 3,
 7,
 39,
 13,
 12,
 13,
 10,
 12,
 1,
 6,
 46,
 7,
 9,
 3,
 15,
 8,
 4,
 13,
 7,
 26,
 6,
 5,
 36,
 3,
 7,
 24,
 3,
 0,
 14,
 5,
 37,
 4,
 42,
 12,
 35,
 13,
 14,
 6,
 10,
 7,
 10,
 7,
 2,
 1,
 24,
 1,
 21,
 5,
 1,
 7,
 7,
 3,
 6,
 11,
 4,
 12,
 5,
 12,
 19,
 12,
 12,
 18,
 11,
 8,
 10,
 8,
 23,
 27,
 15,
 1,
 4,
 6,
 9,
 2,
 6,
 18,
 23,
 13,
 0,
 17,
 15,
 9,
 12,
 8,
 16,
 18,
 6,
 55,
 5,
 9,
 13,
 7,
 7,
 19,
 8,
 12,
 16,
 2,
 20,
 28,
 2,
 29,
 14,
 12,
 25,
 1,
 9,
 11,
 15,
 1,
 8,
 8,
 10,
 23,
 15,
 4,
 4,
 4,
 4,
 8,
 14,
 15,
 6,
 2,
 29,
 7,
 19,
 4,
 17,
 3,
 2,


In [45]:
congress_stats = {}

def new_stat_dict():
    return {
                "mean" : 0,
                "median" : 0,
                "var" : 0,
                "sd" : 0
            }

for i in range(93, 116):
    congress_stats[i] = {
        "house": {
            "bills_sponsored" : new_stat_dict(), 
            "bills_cosponsored" : new_stat_dict(),
            "cosponsors_per_bill" : new_stat_dict(), 
            "cosponsored_sponsored_ratio" : new_stat_dict()
        },
        "senate" : {
            "bills_sponsored" : new_stat_dict(), 
            "bills_cosponsored" : new_stat_dict(),
            "cosponsors_per_bill" : new_stat_dict(), 
            "cosponsored_sponsored_ratio" : new_stat_dict()
        }
    }
    for chamber_key, chamber_val in congress_stats[i].items():
        for field in chamber_val.keys():
            agg = congress_agg[i][chamber_key][field]
            chamber_val[field]["mean"] = np.mean(agg)
            chamber_val[field]["var"] = np.var(agg)
            chamber_val[field]["sd"] = math.sqrt(chamber_val[field]["var"])
            chamber_val[field]["median"] = np.median(agg)

In [46]:
save_json(congress_stats, "data/congress_stats.json")

In [47]:
congress_stats

{93: {'house': {'bills_sponsored': {'mean': 39.663677130044846,
    'median': 31.0,
    'var': 1154.3711918598806,
    'sd': 33.97603849567928},
   'bills_cosponsored': {'mean': 104.3677130044843,
    'median': 79.0,
    'var': 7856.048643648575,
    'sd': 88.63435363135773},
   'cosponsors_per_bill': {'mean': 2.421740870612616,
    'median': 2.0714285714285716,
    'var': 4.683878569853391,
    'sd': 2.1642270143987647},
   'cosponsored_sponsored_ratio': {'mean': 4.420887076873599,
    'median': 2.5,
    'var': 63.76635565541547,
    'sd': 7.985383876521871}},
  'senate': {'bills_sponsored': {'mean': 42.17821782178218,
    'median': 37.0,
    'var': 860.8197235565141,
    'sd': 29.33972943904756},
   'bills_cosponsored': {'mean': 119.11881188118812,
    'median': 118.0,
    'var': 4258.401725321048,
    'sd': 65.25643052850077},
   'cosponsors_per_bill': {'mean': 2.9150286133542327,
    'median': 2.4285714285714284,
    'var': 4.389182717974956,
    'sd': 2.0950376411833167},
   'cosp

In [59]:
# assign z_scores
for leg in leg_map.values():
    congress_map = leg["congress_map"]
    for congress_key, congress_val in congress_map.items():
        chamber = congress_val["chamber"]
        congress_stat = congress_stats[int(congress_key)][chamber]
        for stat_key, stat_val in congress_stat.items():
            if stat_key in congress_val:
                congress_val["z_{}".format(stat_key)] = (congress_val[stat_key] - stat_val["mean"]) / stat_val["sd"]

In [71]:
list(leg_map.values())[0]["switch_congresses"]

[10]

In [66]:
for leg in leg_map.values():
    congress_map = leg["congress_map"]
    congress_vals = list(congress_map.values())
    for i in range(len(congress_vals)):
        congress_vals[i]["experience"] = i

In [87]:
for leg in leg_map.values():
    congress_map = leg["congress_map"]
    # the congresses where the legislator's congress is different from the previous congress
    switch_congresses = []
    for congress_key, congress_val in congress_map.items():
        prev_congress = str(int(congress_key) - 1)
        if prev_congress in congress_map and congress_val["chamber"] != congress_map[prev_congress]["chamber"]:
            switch_congresses.append(congress_key)
    leg["switch_congresses"] = switch_congresses

In [91]:
switchers = [leg for leg in leg_map.values() if len(leg["switch_congresses"]) == 1]

{'104': {'chamber': 'house',
  'congress': 104,
  'bills_sponsored': 4,
  'total_cosponsors': 39,
  'bills_cosponsored': 165,
  'cosponsors_per_bill': 9.75,
  'cosponsored_sponsored_ratio': 41.25,
  'z_bills_sponsored': -0.6536807967930854,
  'z_bills_cosponsored': 0.3526160257258243,
  'z_cosponsors_per_bill': -0.395671246660117,
  'z_cosponsored_sponsored_ratio': 0.4509702693902808,
  'experience': 0},
 '105': {'chamber': 'house',
  'congress': 105,
  'bills_sponsored': 4,
  'total_cosponsors': 31,
  'bills_cosponsored': 184,
  'cosponsors_per_bill': 7.75,
  'cosponsored_sponsored_ratio': 46.0,
  'z_bills_sponsored': -0.6958151952291858,
  'z_bills_cosponsored': -0.02872204533035123,
  'z_cosponsors_per_bill': -0.6691428251880244,
  'z_cosponsored_sponsored_ratio': 0.5077132085869186,
  'experience': 1},
 '106': {'chamber': 'house',
  'congress': 106,
  'bills_sponsored': 6,
  'total_cosponsors': 56,
  'bills_cosponsored': 153,
  'cosponsors_per_bill': 9.333333333333334,
  'cosponsor

[{'chamber': 'house',
  'congress': 100,
  'bills_sponsored': 3,
  'total_cosponsors': 29,
  'bills_cosponsored': 159,
  'cosponsors_per_bill': 9.666666666666666,
  'cosponsored_sponsored_ratio': 53.0,
  'experience': 0},
 {'chamber': 'house',
  'congress': 101,
  'bills_sponsored': 14,
  'total_cosponsors': 122,
  'bills_cosponsored': 154,
  'cosponsors_per_bill': 8.714285714285714,
  'cosponsored_sponsored_ratio': 11.0,
  'experience': 1},
 {'chamber': 'house',
  'congress': 102,
  'bills_sponsored': 16,
  'total_cosponsors': 363,
  'bills_cosponsored': 151,
  'cosponsors_per_bill': 22.6875,
  'cosponsored_sponsored_ratio': 9.4375,
  'experience': 2},
 {'chamber': 'house',
  'congress': 103,
  'bills_sponsored': 14,
  'total_cosponsors': 121,
  'bills_cosponsored': 99,
  'cosponsors_per_bill': 8.642857142857142,
  'cosponsored_sponsored_ratio': 7.071428571428571,
  'experience': 3},
 {'chamber': 'house',
  'congress': 104,
  'bills_sponsored': 13,
  'total_cosponsors': 621,
  'bills_

In [100]:
update_all(engine, tables["leg"], leg_map, verbose = True, increment = 100)

Finished 100 of 2438
Finished 200 of 2438
Finished 300 of 2438
Finished 400 of 2438
Finished 500 of 2438
Finished 600 of 2438
Finished 700 of 2438
Finished 800 of 2438
Finished 900 of 2438
Finished 1000 of 2438
Finished 1100 of 2438
Finished 1200 of 2438
Finished 1300 of 2438
Finished 1400 of 2438
Finished 1500 of 2438
Finished 1600 of 2438
Finished 1700 of 2438
Finished 1800 of 2438
Finished 1900 of 2438
Finished 2000 of 2438
Finished 2100 of 2438
Finished 2200 of 2438
Finished 2300 of 2438
Finished 2400 of 2438
